# From ODE footprints, create a GeoJSONs

[GeoJSON] nowadays are broadly used not only on GIS suite (e.g, QGIS) but also to ingest data in spatial databases. It is very reasonable, then, to have one done for our footprints' ingestion on a database (e.g, [MongoDB]) or for quick visualization (e.g, [Leaflet]).

The documents (i.e, features) in our GeoJSON should have the following fields:

- geometry
- productId
- datasetId
- UTCstart
- UTCEnd
- productType
- path image
- path header

, those are the attributes expected by MEEO's database.

[GeoJSON]: https://en.wikipedia.org/wiki/GeoJSON
[Leaflet]: https://leafletjs.com/examples/geojson/
[MongoDB]: https://docs.mongodb.com/manual/reference/geojson/

In [1]:
import geopandas as gpd
import pandas as pd
pd.options.display.max_columns = 100

## MEX/HRSC

In [2]:
mex = gpd.read_file('mars_mex_hrsc_refdr3_c0a/mars_mex_hrsc_refdr3_c0a.shp')

In [3]:
mex.iloc[0]

CenterLat                                              -38.9095
CenterLon                                               90.5398
MaxLat                                                  -31.035
MinLat                                                  -46.769
EastLon                                                  92.022
WestLon                                                  87.875
EmAngle                                                      -1
InAngle                                                      -1
PhAngle                                                      -1
SolLong                                                      -1
NPoleState                                                    F
SPoleState                                                    F
Target                                                     MARS
ProductId                                    H0010_0000_BL3.JP2
DatasetId                  MEX-M-HRSC-5-REFDR-MAPPROJECTED-V3.0
InstHostId                              

In [4]:
print(mex.ExtURL.iloc[0])
print(mex.ProdURL.iloc[0])
print(mex.FilesURL.iloc[0])
print(mex.LabelURL.iloc[0])

https://ode.rsl.wustl.edu/mars/indexproductpage.aspx?product_id=H0010_0000_BL3.JP2&product_idGeo=25112608
https://ode.rsl.wustl.edu/mars/indexproductpage.aspx?product_id=H0010_0000_BL3.JP2&product_idGeo=25112608
https://ode.rsl.wustl.edu/mars/productfiles.aspx?product_id=H0010_0000_BL3.JP2&product_idGeo=25112608
https://pds-geosciences.wustl.edu/mex/mex-m-hrsc-5-refdr-mapprojected-v3/mexhrs_1003/data/0010/h0010_0000_bl3.lbl


In [5]:
columns = ['geometry', 'ProductId', 'DatasetId', 'InstHostId', 'InstId', 'UTCstart', 'UTCend', 'ProdType']

In [6]:
df = mex[columns]

In [7]:
df.iloc[0]

geometry      POLYGON ((87.875 -46.68, 87.93300000000001 -46...
ProductId                                    H0010_0000_BL3.JP2
DatasetId                  MEX-M-HRSC-5-REFDR-MAPPROJECTED-V3.0
InstHostId                                                  MEX
InstId                                                     HRSC
UTCstart                                2004-01-10T13:52:00.489
UTCend                                  2004-01-10T13:56:37.552
ProdType                                                 REFDR3
Name: 0, dtype: object

In [8]:
df.to_file('mars_mex_hrsc_refdr3_c0a.geojson', driver='GeoJSON')

## MRO/CTX

In [9]:
def shapefile2geojson(shapefile, geojson, columns=None, rows=None):
    '''
    Transform Shapefile to GeoJSON, write it and return GeoDataFrame

    # 'rows' not implemented yet, 
    # the idea is to accept integer (for # top lines) or list of integers (for the line numbers).
    '''
    import geopandas as gpd
    
    shp = gpd.read_file(shapefile)
    df = shp[columns] if columns else shp
    df.to_file(geojson, driver='GeoJSON')
    return df

In [10]:
from os.path import basename

columns = ['geometry', 'ProductId', 'DatasetId', 'InstHostId', 'InstId', 'UTCstart', 'UTCend', 'ProdType']
# missing
# - 'path_image'
# - 'path_label

# input
shapefile = 'mars_mro_ctx_edr_c0a/mars_mro_ctx_edr_c0a.shp'

# output geojson file here
geojson = basename(shapefile).replace('.shp','.geojson')

ctx = shapefile2geojson(shapefile, geojson, columns)

In [11]:
ctx.head()

,geometry,ProductId,DatasetId,InstHostId,InstId,UTCstart,UTCend,ProdType
0,"POLYGON ((-55.26000 -21.68670, -53.84000 -14.5...",MOI_000009_0186_XI_18S051W,MRO-M-CTX-2-EDR-L0-V1.0,MRO,CTX,2006-03-24T04:28:07.724,2006-03-24T04:34:13.285,EDR
1,"POLYGON ((-63.11000 -58.78100, -57.16000 -31.8...",MOI_000009_0438_XN_43S057W,MRO-M-CTX-2-EDR-L0-V1.0,MRO,CTX,2006-03-24T04:41:07.728,2006-03-24T04:53:18.850,EDR
2,"POLYGON ((164.78000 -44.11190, 164.87000 -44.6...",T01_000801_1353_XN_44S195W,MRO-M-CTX-2-EDR-L0-V1.0,MRO,CTX,2006-09-27T21:58:33.012,2006-09-27T21:58:42.621,EDR
3,"POLYGON ((158.10000 7.12620, 158.21000 6.19700...",T01_000801_1867_XN_06N202W,MRO-M-CTX-2-EDR-L0-V1.0,MRO,CTX,2006-09-27T22:14:19.059,2006-09-27T22:14:36.355,EDR
4,"POLYGON ((140.82000 78.51880, 141.26000 78.124...",T01_000801_2584_XN_78N219W,MRO-M-CTX-2-EDR-L0-V1.0,MRO,CTX,2006-09-27T22:37:04.430,2006-09-27T22:37:12.117,EDR


## MRO/HiRISE

In [12]:
shapefile = 'mars_mro_hirise_rdr_c0a/mars_mro_hirise_rdr_c0a.shp'
geojson = basename(shapefile).replace('.shp','.geojson')

hirise = shapefile2geojson(shapefile, geojson, columns)

In [13]:
hirise.head()

,geometry,ProductId,DatasetId,InstHostId,InstId,UTCstart,UTCend,ProdType
0,"POLYGON ((142.55300 -46.80300, 143.39100 -46.8...",AEB_000002_0100_RED,MRO-M-HIRISE-3-RDR-V1.0,MRO,HIRISE,2006-03-25T16:18:52.639,2006-03-25T16:19:09.646,RDR
1,"POLYGON ((155.98300 8.65400, 155.88800 8.64200...",PSP_001804_1890_RED,MRO-M-HIRISE-3-RDR-V1.0,MRO,HIRISE,2006-12-15T00:29:16.882,2006-12-15T00:29:23.845,RDR
2,"POLYGON ((-158.37900 -6.72600, -158.39700 -6.7...",PSP_002343_1735_COLOR,MRO-M-HIRISE-3-RDR-V1.0,MRO,HIRISE,2007-01-26T00:24:20.300,2007-01-26T00:24:23.904,RDR
3,"POLYGON ((-158.34300 -6.72200, -158.43400 -6.7...",PSP_002343_1735_RED,MRO-M-HIRISE-3-RDR-V1.0,MRO,HIRISE,2007-01-26T00:24:20.368,2007-01-26T00:24:23.776,RDR
4,"POLYGON ((61.29800 22.75100, 61.19400 22.73900...",PSP_004471_2030_RED,MRO-M-HIRISE-3-RDR-V1.0,MRO,HIRISE,2007-07-10T20:08:42.980,2007-07-10T20:08:51.989,RDR
